<a href="https://colab.research.google.com/github/MutonyiLewis/DiabetesRecipes/blob/main/GIRidgeRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).





##Import Libraries


In [45]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

##Preview the data


In [46]:
gl_datapath = '/content/drive/MyDrive/Attachment Folder/FoodsWithGIs.csv'
gl_df = pd.read_csv(gl_datapath)
gl_df.head(10)

,Id,Food ID,Calories(kcal),Total Carbohydrate (g),Dietary Fiber (g),Sugars,Protein (g),Fat (g),Glucose,Mixed meal,Gl (%),GL,High Gl (%),Low GI (%),High GL
0,1,Corndog,210,26,2.0,7,5,10.0,2754 ± 762,1424 ± 401,53 ± 14,13 ± 3,67,39,16
1,2,Roasted mixed grain powde,125,23,55.0,3,8,1.3,2747 ± 1320,1372 ± 491,54 ± 18,9 ± 3,72,36,12
2,3,Chocolate cereal bar with nuts,246,25,4.0,14,8,14.0,2744 ± 442,1936 ± 272,72± 14,15 ± 3,86,58,18
3,4,Sweet potato salad,210,25,1.0,15,1,12.0,2877 ± 251,1574 ± 542,55 ± 18,13 ± 4,73,37,17
4,5,Pork and burdock root fried rice,175,26,2.0,3,11,3.0,3376 ± 160,1990 ± 129,60±24,15±6,84,36,21
5,6,Strawberry smoothie,100,25,2.0,22,1,0.0,2898 ± 972,2049 ± 684,67 ± 13,15 ±3,80,54,18
6,7,Crispy tofu in garlic teriyaki,230,26,6.0,10,18,7.0,3077 ± 858,499 ± 194,18±70,4 ± 1,88,-52,5
7,8,Sweet pumpkin and chestnut soup,180,25,0.0,15,4,7.0,3533 ± 791,1328 ± 399,37 ± 6,9±1,43,31,10
8,9,Lentils and Rice with braised tofu,340,50,5.0,6,19,8.0,4139+ 115,2583 ± 142,61 ± 21,27 ± 9,82,40,36
9,10,Spaghetti with tomato meat sauce and cheese,305,50,2.0,8,11,7.0,4075 ± 122,2044 ± 117,49 ± 19,24 ± 9,68,30,33


##Train test split

In [47]:
# Load the training data
X = gl_df.drop(columns=['Id','Food ID','Sugars','Glucose','Mixed meal','Gl (%)', 'GL','High Gl (%)', 'Low GI (%)','High GL'])
y = gl_df['High Gl (%)']

##Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

##Define hyperparameters

In [48]:
# Define the hyperparameter grid
param_grid = {
    "alpha": [0.01, 0.1, 1, 10, 100]
}

##Initialize Model

In [49]:
model = Ridge()

##Perform Grid search

In [50]:
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=Ridge(),
             param_grid={'alpha': [0.01, 0.1, 1, 10, 100]})

##Get the best model

In [51]:
best_model = grid_search.best_estimator_

##Model evaluation

In [52]:
y_pred = best_model.predict(X)
score = np.mean(np.square(y - y_pred))
print("Mean squared error on test set:", score)

Mean squared error on test set: 346.13424513680945


##Use model to train new dataset

In [53]:
recipes_dataset = pd.read_csv('/content/drive/MyDrive/Attachment Folder/RecipesImp.csv')
recipes_dataset.head(3)

df = pd.DataFrame(recipes_dataset)
df = df.drop(columns=['title','index','page','about','ingridients','preparation', 'nutrition per 100g of recipe','vitamin A(mcg)', 'iron(mg)','zinc(mg)'])
df['High Gl (%)'] = ''
y = df['High Gl (%)']
print(y)


0       
1       
2       
3       
4       
      ..
137     
138     
139     
140     
141     
Name: High Gl (%), Length: 142, dtype: object


In [54]:
X1 = pd.DataFrame(df)
X1['Calories(kcal)'] = df['energy(kcal)']
X1['Total Carbohydrate (g)'] = df['carbohydrates(g)']
X1['Dietary Fiber (g)'] = df['fibre(g)']
X1['Protein (g)'] = df['proteins(g)']
X1['Fat (g)'] = df['fat(g)']
X1 = X1.drop(columns=['High Gl (%)','energy(kcal)', 'fibre(g)', 'fat(g)', 'proteins(g)', 'carbohydrates(g)', 'F_factor_est'])
print(X1)
X1.head(3)

     Calories(kcal)  Total Carbohydrate (g)  Dietary Fiber (g)  Protein (g)  \
0             429.0                    52.8                1.6          4.6   
1             413.0                    46.6                2.1          6.0   
2             379.0                    49.9                2.2          7.6   
3             340.0                    48.7                2.1          6.4   
4             443.0                    40.5                3.1         18.8   
..              ...                     ...                ...          ...   
137           337.0                    35.9                1.7          5.9   
138            93.0                    14.5                1.8          1.1   
139           412.0                    39.4                1.1          4.3   
140            60.0                     8.0                0.0          2.0   
141           298.0                     9.6                3.5         10.6   

     Fat (g)  
0       21.8  
1       22.1  
2     

,Calories(kcal),Total Carbohydrate (g),Dietary Fiber (g),Protein (g),Fat (g)
0,429.0,52.8,1.6,4.6,21.8
1,413.0,46.6,2.1,6.0,22.1
2,379.0,49.9,2.2,7.6,16.1


In [55]:
#Add the GL column
y_pred = best_model.predict(X1)
df['High Gl (%)'] = y_pred
print(y_pred)
df.head(45)

[61.06412165 58.60193062 68.7467552  72.51655437 59.57037215 68.52387166
 75.69851337 44.69023532 79.37155505 70.81588109 71.15653438 73.17519558
 71.56026032 65.50620228 72.66155933 71.99841998 73.37210507 72.74937573
 75.55197751 72.92777097 74.42264599 71.60177495 74.57464212 72.90045111
 71.95762283 68.75349794 69.06990592 76.68149923 77.60332977 79.07209801
 76.23815131 76.6964449  81.19514884 74.20610743 75.55987506 77.85326767
 80.96545811 80.25005038 67.26067572 71.59948876 76.02072538 78.64194368
 85.00656754 79.53400745 76.41892245 69.36715867 64.92920011 78.20607938
 77.55786819 71.74889169 74.07693317 79.79848199 65.52998623 72.56844364
 66.73807906 70.3477492  73.1921965  71.64131939 71.78869961 69.17498993
 70.37015005 72.28603106 68.94872066 71.60851076 67.65942355 72.10028664
 62.52481124 50.76529034 60.3564099  55.66613059 72.41456991 46.647782
 56.72476094 44.50142196 69.38261973 66.88314552 56.95630354 41.0106255
 70.81588109 74.72356938 66.89026698 26.69906149 22.50

,energy(kcal),fat(g),carbohydrates(g),proteins(g),fibre(g),F_factor_est,High Gl (%),Calories(kcal),Total Carbohydrate (g),Dietary Fiber (g),Protein (g),Fat (g)
0,429.0,21.8,52.8,4.6,1.6,0.4,61.064122,429.0,52.8,1.6,4.6,21.8
1,413.0,22.1,46.6,6.0,2.1,0.4,58.601931,413.0,46.6,2.1,6.0,22.1
2,379.0,16.1,49.9,7.6,2.2,0.4,68.746755,379.0,49.9,2.2,7.6,16.1
3,340.0,12.9,48.7,6.4,2.1,0.4,72.516554,340.0,48.7,2.1,6.4,12.9
4,443.0,22.2,40.5,18.8,3.1,0.4,59.570372,443.0,40.5,3.1,18.8,22.2
5,325.0,13.1,40.2,8.5,6.2,0.4,68.523872,325.0,40.2,6.2,8.5,13.1
6,270.0,8.5,39.2,8.3,1.7,0.4,75.698513,270.0,39.2,1.7,8.3,8.5
7,449.0,29.8,37.4,7.1,1.4,0.4,44.690235,449.0,37.4,1.4,7.1,29.8
8,241.0,5.6,41.2,5.1,2.8,0.4,79.371555,241.0,41.2,2.8,5.1,5.6
9,0.0,0.0,0.0,0.0,0.0,0.4,70.815881,0.0,0.0,0.0,0.0,0.0


##Add to recipes dataset

In [56]:
recipes_dataset['High GI(%)'] = y_pred
recipes_dataset.head(3)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),proteins(g),fibre(g),vitamin A(mcg),iron(mg),zinc(mg),F_factor_est,High GI(%)
0,Kaimati(Fried Dumplings),15003,24,Kaimatis get their unique flavour from the sty...,"wheat flour, refined\nwater, vanilla essence,...",Put yeast in a small container.\n Add 50ml of...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,4.6,1.6,30,2.1,0.45,0.4,61.064122
1,Mahamri\n(Swahili Doughnut),15004,26,This is a typical traditional recipe among the...,"wheat flour,\ncoconut milk\nwhite sugar\ndry y...","Break the coconut shell, drain the water and...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,6.0,2.1,41,2.8,0.56,0.4,58.601931
2,"Enriched Mandazi \n(East African Doughnuts, En...",15124,28,A popular snack among urban dwellers across th...,self-raising wheat flour\neggs\nmargarine\nRin...,"? Put flour, salt, sugar and lemon rind into ...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,7.6,2.2,90,3.3,0.66,0.4,68.746755


#Calculating the Low GL(%)

In [57]:
gl_datapath = '/content/drive/MyDrive/Attachment Folder/FoodsWithGIs.csv'
gl_df = pd.read_csv(gl_datapath)

# Load the training data
X = gl_df.drop(columns=['Id','Food ID','Sugars','Glucose','Mixed meal','Gl (%)', 'GL','High Gl (%)', 'Low GI (%)','High GL'])
y = gl_df['High Gl (%)']

##Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Define the hyperparameter grid
param_grid = {
    "alpha": [0.01, 0.1, 1, 10, 100]
}

#Load Model
model = Ridge()

#Perform Grid Search
grid_search = GridSearchCV(model, param_grid, cv=5)
grid_search.fit(X_train, y_train)

#Get the best model
best_model = grid_search.best_estimator_

#Evaluate the model
y_pred = best_model.predict(X)
score = np.mean(np.square(y - y_pred))
print("Mean squared error on test set:", score)


Mean squared error on test set: 331.07479659067616


###Use on Recipes dataset

In [60]:
#Load the dataset
df = pd.DataFrame(recipes_dataset)
df = df.drop(columns=['title','index','page','about','ingridients','preparation', 'nutrition per 100g of recipe','vitamin A(mcg)', 'iron(mg)','zinc(mg)','High GI(%)', 'F_factor_est'])
df['Low Gl (%)'] = ''
y = df['Low Gl (%)']


X2 = pd.DataFrame(df)
X2['Calories(kcal)'] = df['energy(kcal)']
X2['Total Carbohydrate (g)'] = df['carbohydrates(g)']
X2['Dietary Fiber (g)'] = df['fibre(g)']
X2['Protein (g)'] = df['proteins(g)']
X2['Fat (g)'] = df['fat(g)']
X2 = X2.drop(columns=['Low Gl (%)','energy(kcal)', 'fibre(g)', 'fat(g)', 'proteins(g)', 'carbohydrates(g)'])

#Add the GL column
y_pred = best_model.predict(X2)
df['Low Gl (%)'] = y_pred
print(y_pred)
df.head(45)

[65.12245996 62.02337186 69.84207807 73.30587357 56.92489583 68.37737559
 74.2651405  49.51838554 78.73022734 70.47168901 68.24909218 72.42630299
 71.09942673 65.97103211 72.0457055  71.40111149 73.07828186 72.24507773
 74.77727446 72.77443777 74.02277304 70.76344767 73.997598   72.3422304
 71.71573453 68.03352325 68.89149309 75.217856   77.20367964 78.34108706
 76.16213249 76.11033969 80.0378853  73.9047137  74.49223061 75.73329005
 76.83823186 77.56120655 68.47434491 69.4747714  74.23778112 75.99666515
 83.23467488 78.40682446 75.89049402 70.02108165 66.4739376  75.81813634
 75.18951304 71.50024047 72.44306909 78.77555005 64.04883531 70.65882815
 65.59374043 69.61958239 73.16215899 70.21608664 69.18137771 67.20734632
 68.22396249 70.77737928 67.42022559 69.70824873 67.2205736  70.44452087
 56.69640718 41.71601076 51.30314825 55.5038101  69.95731903 40.49980314
 55.90262185 30.1442233  67.69449344 61.5279266  52.55768011 33.61819979
 70.47168901 70.92207649 64.12616557 22.1089361  23.

,energy(kcal),fat(g),carbohydrates(g),proteins(g),fibre(g),Low Gl (%),Calories(kcal),Total Carbohydrate (g),Dietary Fiber (g),Protein (g),Fat (g)
0,429.0,21.8,52.8,4.6,1.6,65.122460,429.0,52.8,1.6,4.6,21.8
1,413.0,22.1,46.6,6.0,2.1,62.023372,413.0,46.6,2.1,6.0,22.1
2,379.0,16.1,49.9,7.6,2.2,69.842078,379.0,49.9,2.2,7.6,16.1
3,340.0,12.9,48.7,6.4,2.1,73.305874,340.0,48.7,2.1,6.4,12.9
4,443.0,22.2,40.5,18.8,3.1,56.924896,443.0,40.5,3.1,18.8,22.2
5,325.0,13.1,40.2,8.5,6.2,68.377376,325.0,40.2,6.2,8.5,13.1
6,270.0,8.5,39.2,8.3,1.7,74.265141,270.0,39.2,1.7,8.3,8.5
7,449.0,29.8,37.4,7.1,1.4,49.518386,449.0,37.4,1.4,7.1,29.8
8,241.0,5.6,41.2,5.1,2.8,78.730227,241.0,41.2,2.8,5.1,5.6
9,0.0,0.0,0.0,0.0,0.0,70.471689,0.0,0.0,0.0,0.0,0.0


In [61]:
recipes_dataset['Low GI(%)'] = y_pred
recipes_dataset.head(3)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),proteins(g),fibre(g),vitamin A(mcg),iron(mg),zinc(mg),F_factor_est,High GI(%),Low GI(%)
0,Kaimati(Fried Dumplings),15003,24,Kaimatis get their unique flavour from the sty...,"wheat flour, refined\nwater, vanilla essence,...",Put yeast in a small container.\n Add 50ml of...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,4.6,1.6,30,2.1,0.45,0.4,61.064122,65.122460
1,Mahamri\n(Swahili Doughnut),15004,26,This is a typical traditional recipe among the...,"wheat flour,\ncoconut milk\nwhite sugar\ndry y...","Break the coconut shell, drain the water and...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,6.0,2.1,41,2.8,0.56,0.4,58.601931,62.023372
2,"Enriched Mandazi \n(East African Doughnuts, En...",15124,28,A popular snack among urban dwellers across th...,self-raising wheat flour\neggs\nmargarine\nRin...,"? Put flour, salt, sugar and lemon rind into ...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,7.6,2.2,90,3.3,0.66,0.4,68.746755,69.842078


In [63]:
# export the DataFrame to a CSV file
recipes_dataset.to_csv('RecipesWithGILimits.csv', index=False)